In [22]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem

In [23]:
# Let's read in the file
input_data = pd.read_csv('data/mod_covalent_smiles.csv')

In [24]:
input_data.head()

,Library Plate,Source well,Library Name,SMILES,Modified SMILES,Code,Volume (ul),Concentration (mM),old_ID,covalent
0,1181202-Y4-02,R05,shipment 6 - moonshot,C=C(C(=O)NC=1C=CC=CC1)C=2C=CC=NC2,CC(C(Nc1ccccc1)=O)c1cnccc1,BEN-DND-031a96cc-8,4,100,BEN-DND-031-8,NaN
1,1181202-Y4-02,O06,shipment 6 - moonshot,CN(C(=O)CC1=NOC=2C=CC=CC12)C=3C=CC(NC(=O)OC(C)...,CC(C)(C)OC(Nc(cc1)ccc1N(C)C(Cc1noc2c1cccc2)=O)=O,AAR-POS-8a4e0f60-9,4,100,s_22____12335866____132823,no
2,1181202-Y4-02,P05,shipment 6 - moonshot,CN(C)CC(=C)C(=O)N1CC(C=2C=CC=CC2)C=3C=CC=CC3C1,CC(CN(C)C)C(N(CC1c2ccccc2)Cc2c1cccc2)=O,REN-UFA-aaaed2d6-3,4,100,REN-UFA-aaa-3,NaN
3,1181202-Y4-02,C05,shipment 6 - moonshot,CC=1C=CC(=CC1)S(=O)(=O)NC2CCN(CC2)C(=O)CCl,CC(N(CC1)CCC1NS(c1ccc(C)cc1)(=O)=O)=O,WAR-XCH-79d12f6e-4,4,100,WAR-XCH-79d-4,NaN
4,1181202-Y4-02,E05,shipment 6 - moonshot,ClCC(=O)N1CCN(CC1)S(=O)(=O)C=2C=CSC2,CC(N(CC1)CCN1S(c1cscc1)(=O)=O)=O,MAT-POS-ee51dedd-2,4,100,MAK-UNK-643-10,NaN


In [25]:
compound_mols = [Chem.MolFromSmiles(smi) for smi in input_data.SMILES]

In [26]:
# Let's define some covalent structures that we can use to search for covalents and replace with 
# Make a dictionary of colavent bits (key) and the transformation (value) we'd like
cov_dict = {"[$([#1,*])]-[#7]-[#6](=O)-[#6](-[$([#1,*])])=[#6]":"[#6]-[#6](-[$([#1,*])])-[#6](=O)-[#7]-[$([#1,*])]",
           "[$([#1,*])]-[#7]-[#6](=O)-[#6]=[#6]":"[#6]-[#6]-[#6](=O)-[#7]-[$([#1,*])]",
           "Cl[#6]-[#6](=O)-[#7]-[$([#1,*])]":"[#6]-[#6](=O)-[#7]-[$([#1,*])]",
           "[#6]-[#6](=O)-[#7]-[#6]C#C":"[$([#1,*])]-[#6](=O)-[#7]-[#6]-[#6]=[#6]",
           "[Ar]C#N":"[Ar][#6]=[#7]"}



In [47]:
mod_smiles = []

for mol in compound_mols:
    for key, value in cov_dict.items():
        patt = Chem.MolFromSmarts(key)
        repl = Chem.MolFromSmiles(value)
        try:
            rms = AllChem.ReplaceSubstructs(mol,patt,repl)
            smi = Chem.MolToSmiles(rms)
            mod_smiles.append(smi)
        except:
            pass

In [48]:
mod_smiles

[]

In [29]:
patt = Chem.MolFromSmarts("[$([#1,*])]-[#7]-[#6](=O)-[#6](-[$([#1,*])])=[#6]")
compound_mols[0].HasSubstructMatch(patt)

True

In [30]:
repl = Chem.MolFromSmarts("[#6]-[#6](-[$([#1,*])])-[#6](=O)-[#7]-[$([#1,*])]")

In [32]:
rms  = AllChem.ReplaceSubstructs(compound_mols[0],patt,repl)

In [35]:
Chem.MolToSmiles(rms[0])  

'[*]NC(=O)C([*])c1ccccc1.cccnc'